## How to use this Interactive Campaign Runner
- Select a YAML config (baseline at `examples/configs/baseline.yaml`).
- Override simulation parameters (widths, circuits, shots, backend, seed).
- Choose output directory and a campaign id (auto-filled).
- Click "Run Campaign" to start, or "Resume Only" to continue a prior run.
- Use "Stop" to halt after the current width completes.
- The progress bar updates after each width; aggregated results table is shown below.
Notes: results are written to an HDF5 file under the output folder; plots are created per-width.

# Interactive Campaign Runner
Use this notebook to configure and run multi-width QV campaigns with a simple UI. It supports resume and live progress display.

In [1]:
# Setup imports and sys.path
import sys, os, json, time, threading
from pathlib import Path
from datetime import datetime
from IPython.display import display, clear_output
import ipywidgets as widgets
root = Path.cwd()
src_path = root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
from spinq_qv.config.schemas import Config
from spinq_qv.experiments.campaign import ProductionCampaignRunner

In [2]:
# UI controls
config_path = widgets.Text(value=str(root / "examples" / "configs" / "baseline.yaml"), description='Config YAML:', layout=widgets.Layout(width='80%'))
widths_text = widgets.Text(value='2,3,4', description='Widths:')
n_circuits = widgets.IntText(value=10, description='Circuits:')
n_shots = widgets.IntText(value=500, description='Shots:')
backend = widgets.Dropdown(options=['statevector','density_matrix','mcwf','tensornet'], value='statevector', description='Backend:')
seed = widgets.IntText(value=12345, description='Seed:')
parallel = widgets.Checkbox(value=False, description='Parallel')
workers = widgets.IntSlider(value=2, min=1, max=8, step=1, description='Workers')
output_dir = widgets.Text(value=str(root / "results" / "campaign_ui"), description='Output dir:', layout=widgets.Layout(width='80%'))
campaign_id = widgets.Text(value=f"nb_{datetime.now().strftime('%Y%m%d_%H%M%S')}", description='Campaign ID:')
resume = widgets.Checkbox(value=False, description='Resume')
run_btn = widgets.Button(description='Run Campaign', button_style='success')
resume_btn = widgets.Button(description='Resume Only', button_style='warning')
stop_btn = widgets.Button(description='Stop', button_style='danger')
progress = widgets.IntProgress(value=0, min=0, max=100, description='Progress:')
log_out = widgets.Output()
controls = widgets.VBox([config_path, widths_text, widgets.HBox([n_circuits, n_shots, backend]), widgets.HBox([seed, parallel, workers]), output_dir, widgets.HBox([campaign_id, resume]), widgets.HBox([run_btn, resume_btn, stop_btn]), progress, log_out])
display(controls)

In [3]:
# Campaign runner thread & handlers
stop_flag = {'stop': False}
def build_config():
    cfg = Config.from_yaml(Path(config_path.value))
    # Apply UI overrides
    cfg.simulation.widths = [int(x.strip()) for x in widths_text.value.split(',') if x.strip()]
    cfg.simulation.n_circuits = int(n_circuits.value)
    cfg.simulation.n_shots = int(n_shots.value)
    cfg.simulation.backend = backend.value
    cfg.simulation.random_seed = int(seed.value)
    return cfg
def on_run_clicked(_):
    stop_flag['stop'] = False
    thread = threading.Thread(target=run_campaign, kwargs={'resume_only': False})
    thread.start()
def on_resume_clicked(_):
    stop_flag['stop'] = False
    thread = threading.Thread(target=run_campaign, kwargs={'resume_only': True})
    thread.start()
def on_stop_clicked(_):
    stop_flag['stop'] = True
def run_campaign(resume_only=False):
    log_out.clear_output()
    with log_out:
        try:
            cfg = build_config()
            out_dir = Path(output_dir.value)
            out_dir.mkdir(parents=True, exist_ok=True)
            runner = ProductionCampaignRunner(
                config=cfg,
                output_dir=out_dir,
                campaign_id=campaign_id.value or None,
                resume=resume.value or resume_only,
            )
            widths = cfg.simulation.widths
            done = set(runner.state.completed_widths)
            total = len(widths)
            progress.value = int(100 * len(done)/max(1,total))
            print(f"Starting campaign {runner.campaign_id} in {out_dir}")
            # Run width by width to allow UI progress updates
            for w in widths:
                if stop_flag['stop']:
                    print("Stopped by user.")
                    break
                if w in runner.state.completed_widths:
                    print(f"Width m={w} already completed; skipping.")
                    continue
                print(f"Running width m={w}...")
                runner._run_single_width(
                    w, parallel=parallel.value, n_workers=int(workers.value), enable_profiling=False
)
                runner.state.mark_completed(w)
                done.add(w)
                progress.value = int(100 * len(done)/max(1,total))
            print("Done. Results:", runner.state.results_file)
        except Exception as e:
            print("Error:", e)
run_btn.on_click(on_run_clicked)
resume_btn.on_click(on_resume_clicked)
stop_btn.on_click(on_stop_clicked)

Error: [Errno 2] No such file or directory: 'c:\\Users\\20232788\\Desktop\\Year3\\IQT\\Group Project\\spinq_qv_sim\\notebooks\\examples\\configs\\baseline.yaml'
Error: [Errno 2] No such file or directory: 'c:\\Users\\20232788\\Desktop\\Year3\\IQT\\Group Project\\spinq_qv_sim\\notebooks\\examples\\configs\\baseline.yaml'
Error: [Errno 2] No such file or directory: 'c:\\Users\\20232788\\Desktop\\Year3\\IQT\\Group Project\\spinq_qv_sim\\notebooks\\examples\\configs\\baseline.yaml'
Starting campaign nb_20251017_012422 in c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui
Running width m=2...
Starting campaign nb_20251017_012422 in c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui
Running width m=2...
[OK] Results saved to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\qv_run_20251017_012612.h5

Generating plots in c:\Users\20232788\Desktop\Year3\IQT\Gro

C:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\src\spinq_qv\analysis\plots.py:288: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Saved summary plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_012612\qv_summary.png
Saved distribution plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_012612\hop_dist_m2.png
Saved HOP plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_012612\hop_vs_width.svg
Saved summary plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_012612\qv_summary.svg
Saved HOP plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_012612\hop_vs_width.svg
Saved summary plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_012612\qv_summary.svg
Saved distribution plot to c:\Users\20

In [4]:
# Visualization helper: show current aggregated results
import h5py
from glob import glob
from IPython.display import HTML
def show_aggregated(h5_path):
    with h5py.File(h5_path, 'r') as f:
        if 'aggregated' not in f:
            print('No aggregated group yet')
            return
        rows=[]
        for w in sorted(f['aggregated'].keys(), key=lambda x: int(x)):
            attrs = dict(f['aggregated'][w].attrs)
            rows.append((int(w), attrs.get('mean_hop'), attrs.get('ci_lower'), attrs.get('ci_upper'), attrs.get('std_hop'), attrs.get('pass_qv')))
    import pandas as pd
    df = pd.DataFrame(rows, columns=['width','mean_hop','ci_lower','ci_upper','std_hop','pass_qv'])
    display(df)
def find_latest_h5():
    files = sorted(glob(str(Path(output_dir.value) / "**/*.h5"), recursive=True))
    return files[-1] if files else None
latest = find_latest_h5()
if latest:
    print('Latest HDF5:', latest)
    show_aggregated(latest)
else:
    print('No HDF5 found yet. Run a campaign.')

Latest HDF5: c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_4\qv_run_20251017_012619.h5


,width,mean_hop,ci_lower,ci_upper,std_hop,pass_qv
0,4,0.8402,0.8164,0.8614,None,True


Starting campaign nb_20251017_012422 in c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui
Running width m=2...
[OK] Results saved to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\qv_run_20251017_013059.h5

Generating plots in c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_013059...
[OK] Results saved to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\qv_run_20251017_013059.h5

Generating plots in c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_013059...
Saved HOP plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_013059\hop_vs_width.png
Saved HOP plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\resul

C:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\src\spinq_qv\analysis\plots.py:288: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Saved summary plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_013059\qv_summary.png
Saved distribution plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_013059\hop_dist_m2.png
Saved HOP plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_013059\hop_vs_width.svg
Saved HOP plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_013059\hop_vs_width.svg
Saved summary plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_013059\qv_summary.svg
Saved distribution plot to c:\Users\20232788\Desktop\Year3\IQT\Group Project\spinq_qv_sim\notebooks\results\campaign_ui\width_2\plots_20251017_013059\hop_dist_m2.svg
[OK] All plots saved to c:\Users

### Tips and troubleshooting
- If imports fail, ensure `src/` is on `sys.path` (cell 2).
- If plots/aggregates are missing, run at least one width first.
- Resume requires the campaign state JSON to exist in the output directory.